In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from keras.models import Sequential
from keras.layers import Dense, Input

from sklearn.metrics import confusion_matrix, accuracy_score

In [3]:
df=pd.read_csv('HIGGS_8K.csv',header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,1.0,0.869293,-0.635082,0.225690,0.327470,-0.689993,0.754202,-0.248573,-1.092064,0.000000,...,-0.010455,-0.045767,3.101961,1.353760,0.979563,0.978076,0.920005,0.721657,0.988751,0.876678
1,0.0,1.630428,0.404143,0.401026,2.722409,0.355644,1.362199,0.015818,-1.174111,0.000000,...,0.387635,1.596321,1.550981,0.859235,0.827233,0.995416,0.764766,0.520597,0.850413,0.929865
2,1.0,0.326305,-0.797735,0.223471,1.248234,-1.427857,0.923767,-0.278280,-0.470052,0.000000,...,-0.426866,1.153501,0.000000,0.887944,0.924601,0.987189,0.854094,0.385013,0.789912,0.724627
3,1.0,1.381170,0.521993,0.574698,0.357347,0.037136,0.413057,-0.607036,-1.308270,0.000000,...,0.567525,-1.475778,0.000000,0.936186,1.107217,0.983808,0.693327,0.562045,0.825387,0.846233
4,1.0,0.304161,-0.736375,0.686225,0.477172,-1.274736,0.907003,0.324770,-0.463954,2.173076,...,0.904818,1.165154,0.000000,1.181790,1.033701,0.993943,0.648399,0.936097,0.732592,0.661779


In [5]:
# While examining the data, some string elements were noticed
print(df.dtypes[df.dtypes != 'float64'])

17    object
dtype: object


In [7]:
# After locating the issue, we fix it 
print(f'String value: {df.iloc[0,17]} (type: {type(df.iloc[0, 17])})')
df.iloc[:, 17] = pd.to_numeric(df.iloc[:, 17], errors='coerce').fillna(0) 
print(f'Fixed value: {df.iloc[0, 17]} (type: {type(df.iloc[0, 17])})')

String value: 0.000000000000000000e+00.1 (type: <class 'str'>)
Fixed value: 0.0 (type: <class 'float'>)


In [9]:
# Separation of the data, according to the instructions
classification=df.iloc[:,0]
low_level=df.iloc[:,1:21]
high_level=df.iloc[:,22:]

### High level

In [58]:
# Splitting the dataset
high_level_train, high_level_test, classification_train, classification_test = train_test_split(high_level, classification, test_size=0.25, random_state=42)

# Feature scaling
scaler = StandardScaler()
high_level_train_scaled = scaler.fit_transform(high_level_train)
high_level_test_scaled =scaler.transform(high_level_test)

In [60]:
# Building the ANN
ann = tf.keras.models.Sequential()

# Input layer
ann.add(Input(shape=(7,)))
ann.add(tf.keras.layers.Dense(12,activation='relu'))

#Second hidden layer
ann.add(tf.keras.layers.Dense(8, activation='relu'))

# Output layer
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [62]:
# Training the ANN

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                     │ (None, 12)                  │              96 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 8)                   │             104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 209 (836.00 B)

 Trainable params: 209 (836.00 B)

 Non-trainable params: 0 (0.00 B)

In [64]:
ann.fit(high_level_train_scaled, classification_train, batch_size=32, epochs=200)

Epoch 1/200
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5326 - loss: 0.6902
Epoch 2/200
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6120 - loss: 0.6726
Epoch 3/200
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6310 - loss: 0.6574
Epoch 4/200
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6404 - loss: 0.6460
Epoch 5/200
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6463 - loss: 0.6318
Epoch 6/200
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6556 - loss: 0.6247
Epoch 7/200
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6601 - loss: 0.6226
Epoch 8/200
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6585 - loss: 0.6155
Epoch 9/200
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6792 - loss: 0.6013
Epoch 10/200
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6647 - loss: 0.6026
Epoch 11/200
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6753 - loss: 0.6041
Epoch 12/200
188/188 ━━━━━━━━━━━━━━━━━━━━

In [66]:
# Predicting the test set results
classification_pred=ann.predict(high_level_test_scaled)
classification_pred=(classification_pred>0.5)

# Making the confusion matrix
conf_matr = confusion_matrix(classification_test, classification_pred)
print(f'Confusion matrix for the low level is: \n{conf_matr}')
accuracy_score(classification_test, classification_pred)
print(f'Accuracy score is: {accuracy_score(classification_test, classification_pred)}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Confusion matrix for the low level is: 
[[659 319]
 [279 744]]
Accuracy score is: 0.7011494252873564


In [68]:
# Evaluate the model on test data
loss, accuracy = ann.evaluate(high_level_test_scaled, classification_test)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6997 - loss: 0.5790
Test loss: 0.5754027962684631
Test accuracy: 0.7011494040489197
